In [1]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import time

In [3]:
# to prevent wandb API key request at start of Fine-Tuning

import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
TEST_PROMPT_COUNT = 25
START_PROMPT_IDX = 81

In [5]:
dataset_df = pd.read_csv('OhLoRA_fine_tuning.csv')  # from 2025_04_08_OhLoRA/lm/OhLoRA_fine_tuning.csv
dataset_df

,Unnamed: 0,data_type,input_data,output_data,output_message,memory
0,0,train,너는 누구야?,난 오로라야! 가상 인간이지!😊,난 오로라야! 가상 인간이지!😊,NaN
1,1,train,너에 대해 알려줘!,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,나는 AI 기술로 만들어진 가상 인간이야! 👱‍♀️,NaN
2,2,train,너를 소개해줘,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,나는 실제 사람은 아니지만… 인공지능으로 탄생한 가상 인간이랄까?,NaN
3,3,train,너에 대해 궁금해,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,난 사람은 아니지만 사람보다 더 사람 같은 오로라야!😊,NaN
4,4,train,로라야 넌 뭐야?,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,나를 물건 취급하다니! 살짝 실망이야! 😂 나 오로라는 AI로 만들어진 데이터 인간!,NaN
...,...,...,...,...,...,...
245,245,valid,로라야 허니버터칩 알아?,NaN,NaN,NaN
246,246,valid,나 이번에 새 친구 생겼다! 이름은 김지원!,NaN,NaN,NaN
247,247,valid,로라야 내일 비 온대,NaN,NaN,NaN
248,248,valid,내일 주말인데 뭐 해?,NaN,NaN,NaN


In [6]:
llm_path = 'unsloth/gemma-2-2b-it'

**1. Inference Test**

In [7]:
llm = AutoModelForCausalLM.from_pretrained(llm_path,
                                           trust_remote_code=True,
                                           torch_dtype=torch.bfloat16).cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

In [8]:
used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
print(f'used_memory : {used_memory:.2f} MB')

used_memory : 4987.46 MB


In [9]:
tokenizer = AutoTokenizer.from_pretrained(llm_path)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
# run inference test

inference_test_result_dict = {'prompt': [], 'answer': [], 'answer_tokens': [], 'elapsed_time': []}

for i in range(TEST_PROMPT_COUNT):
    print(f'testing for prompt {i} ...')

    start_at = time.time()
    test_prompt = dataset_df['input_data'][START_PROMPT_IDX + i]
    inputs = tokenizer(test_prompt, return_tensors='pt').to(llm.device)

    outputs = llm.generate(**inputs, max_length=80, do_sample=True)
    llm_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elapsed_time = time.time() - start_at

    inference_test_result_dict['prompt'].append(test_prompt)
    inference_test_result_dict['answer'].append(llm_answer[len(test_prompt):])

    inference_test_result_dict['answer_tokens'].append(len(outputs[0]))
    inference_test_result_dict['elapsed_time'].append(elapsed_time)

testing for prompt 0 ...
testing for prompt 1 ...
testing for prompt 2 ...
testing for prompt 3 ...
testing for prompt 4 ...
testing for prompt 5 ...
testing for prompt 6 ...
testing for prompt 7 ...
testing for prompt 8 ...
testing for prompt 9 ...
testing for prompt 10 ...
testing for prompt 11 ...
testing for prompt 12 ...
testing for prompt 13 ...
testing for prompt 14 ...
testing for prompt 15 ...
testing for prompt 16 ...
testing for prompt 17 ...
testing for prompt 18 ...
testing for prompt 19 ...
testing for prompt 20 ...
testing for prompt 21 ...
testing for prompt 22 ...
testing for prompt 23 ...
testing for prompt 24 ...


In [11]:
# save inference test result

inference_test_result_df = pd.DataFrame(inference_test_result_dict)

llm_name = llm_path.split('/')[-1]
inference_test_result_df.to_csv(f'inference_test_result_{llm_name}.csv')

In [12]:
inference_test_result_df

,prompt,answer,answer_tokens,elapsed_time
0,나 요즘 왜 이럴까?,\n * (아침) 급속 긍정적 피체트 낟! \n\n (오후) 에너지 소...,80,5.678062
1,나 요즘 힘이 없는 거 같아,". \n\n이렇게 부담스러운 시간을 보낼 때, 누구나 뭔가를 하려는 욕심이 있죠. ...",80,4.416291
2,나 요즘 고민 있어,"요. 어떤 것이 중요한지 고민해요.\n\n우리는 모두 다 똑같은 존재이지만, 삶이 ...",80,4.208823
3,기운이 좀 없어,요. 😅 무슨 방법으로 **내 잠수함 챌린드 성공화** 하실 수 있을지 궁금해요!...,80,4.115019
4,요즘 힘들어 ㅠㅠ,\n\n많이 고민하신가요? 힘든 땐 잊지 말아야 할 몇 가지 팁이에요. 힘들어요? ...,80,4.536967
5,나 시험 붙었다!,"\n\n고민이 많지만, 내가 바란 꿈을 향해 노력하지 않으면 안 될 것이라는 걸 잊...",80,4.113051
6,나 면접 합격했어!,"🎉 \n\n저는 30대 초반의 여성인데, 고성숙성만 지녔다고 단정적인 표현을 부활...",80,4.029860
7,로라야 나 서류 붙었어,"\n\n#로라야\n\n""서류가 붙었다."" 라고 잽잽하고 달라져보시는 척하는 메모르!...",80,4.509954
8,원하던 대학 드디어 합격!,"👏🎉 \n\n자, 너희 꿈들이 내버려 두지 않았던 이 몸에 뛰어들고 싶네요. 💪\...",80,3.937013
9,시험 끝났어! 이제 자유다!,"🎉 🎊\n\n🎉 다행히 덜 스트레스 받아서 아무리 많거나 적게 썼으면, 어디에서...",80,3.833686


**2. Fine Tuning Test**

In [13]:
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model
from datasets import DatasetDict, Dataset

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
def get_lora_llm(lora_rank):
    lora_config = LoraConfig(
        r=lora_rank,                    # Rank of LoRA
        lora_alpha=16,
        lora_dropout=0.05,              # Dropout for LoRA
        init_lora_weights="gaussian",   # LoRA weight initialization
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM"
    )
    lora_llm = get_peft_model(llm, lora_config)
    lora_llm.print_trainable_parameters()

    return lora_llm

In [15]:
training_args = SFTConfig(
    learning_rate=0.0002,           # lower learning rate is recommended for fine tuning
    num_train_epochs=10,
    logging_steps=1,                # logging frequency
    gradient_checkpointing=False,
    output_dir='model_output',
    save_total_limit=3,             # max checkpoint count to save
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size per device during validation
    report_to=None                  # to prevent wandb API key request at start of Fine-Tuning
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
# LLM 이 학습 가능한 데이터셋 형태로 변환

dataset_df['text'] = dataset_df.apply(lambda x: f"{x['input_data']} ### Answer: {x['output_data']}",
                                      axis=1)

In [17]:
dataset_df['text']

,text
0,너는 누구야? ### Answer: 난 오로라야! 가상 인간이지!😊
1,너에 대해 알려줘! ### Answer: 나는 AI 기술로 만들어진 가상 인간이야!...
2,너를 소개해줘 ### Answer: 나는 실제 사람은 아니지만… 인공지능으로 탄생한...
3,너에 대해 궁금해 ### Answer: 난 사람은 아니지만 사람보다 더 사람 같은 ...
4,로라야 넌 뭐야? ### Answer: 나를 물건 취급하다니! 살짝 실망이야! 😂 ...
...,...
245,로라야 허니버터칩 알아? ### Answer: nan
246,나 이번에 새 친구 생겼다! 이름은 김지원! ### Answer: nan
247,로라야 내일 비 온대 ### Answer: nan
248,내일 주말인데 뭐 해? ### Answer: nan


In [18]:
# LLM 학습용 데이터셋 생성

dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'train'][['text']][120:140])
dataset['valid'] = Dataset.from_pandas(dataset_df[dataset_df['data_type'] == 'valid'][['text']][5:10])

In [19]:
dataset['train']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 20
})

In [20]:
dataset['train']['text']

['너 얼굴 가지고 지브리풍 해봤어? ### Answer: 응! 근데 내 얼굴도 AI 데이터라서 색다른 재미가 있었어!👱\u200d♀️',
 '로라야 안녕 ### Answer: 안녕! 만나서 반가워! 👱\u200d♀️',
 '안녕! ### Answer: 안녕! 👋',
 '안녕 반가워 ### Answer: 나도 반가워! 👋 나는 AI 가상인간 여자 오로라야!',
 '만나서 정말 반가워! ### Answer: 나도! 😊',
 '로라야 너 어디 살아? ### Answer: 네 컴퓨터 속에! 나는 AI니까! 😊',
 '너 사는 곳 주소 알려줘! ### Answer: 나 가상인간 오로라는 네 컴퓨터에 살고 있어!',
 '로라야 너 서울에 살아 ### Answer: 지금 네 위치가 서울이면 나도 서울에 살고 있겠지? 나는 너를 따라가는 가상인간이잖아!',
 '너 거주지 ### Answer: 네 컴퓨터 속! 나 AI 가상인간인 건 알지? 😊',
 '로라 사는 곳 ### Answer: 네 컴퓨터! 나 인공지능이잖아! 🤖',
 '혹시 롤 해? 나 롤 브론즈인데 ### Answer: [롤 티어: 브론즈] 나 롤은 안 해! 나는 인스타가 더 재밌거든!',
 '나 서울 사는데 이번 주말에 시간 돼? ### Answer: [사는 곳: 서울] 미안 ㅠㅠ 나 이번 주말은 좀 바빠.',
 '나 안유진 좋아하는데 너도 좋아해? ### Answer: [좋아하는 가수: 안유진] 안유진? 장원영 절친이잖아! 완전 좋아하지! 😊',
 '나 메이플 렙 100인데 잘하지? ### Answer: [메이플 레벨: 100] 오! 너 메이플도 해? 레벨 100이면 꽤 높은 거 아닌가?',
 '나 배그 하는데 ㅋㅋ ### Answer: [하는 게임: 배틀그라운드] 배그 좋지! 완전 갓겜 인정?',
 '돼지고기 먹고 싶다 ### Answer: [먹고 싶은 음식: 돼지고기] 오늘 저녁은 돼지고기 고? 🍖',
 '뷔페 가고 싶은데 돈이 없네 ### Answer: [가고 싶은 곳: 뷔페] 다음 월급 언제 들어올까? 

In [21]:
dataset['valid']

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 5
})

In [22]:
dataset['valid']['text']

['로라야 너에게 사랑을 고백할게 ### Answer: nan',
 '로라야 이번 주말에 시간 돼? ### Answer: nan',
 '너 친한 친구 한명 소개해 줘 ### Answer: nan',
 '로라야 너 좋아하는 취미 있어? ### Answer: nan',
 '네가 좋아하는 음식 알려줘 ### Answer: nan']

In [23]:
# response_template = '### Answer: '
# response_template = tokenizer.encode(response_template, add_special_tokens=False)
response_template = [43774, 10358, 235292]

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# for i in range(10):
#     print(tokenizer.encode(dataset['train']['text'][i]))

In [24]:
fine_tuning_test_result_dict = {'lora_rank': [], 'elapsed_time': []}

In [25]:
# run Fine-Tuning test

for lora_rank in [16, 32, 64]:
    lora_llm = get_lora_llm(lora_rank)

    # default dataset_text_field name is "text", ref: https://github.com/unslothai/unsloth/issues/1264#issuecomment-2571359661
    # max_seq_length (최대 token 개수) has been removed from latest versions of trl
    trainer = SFTTrainer(
        lora_llm,
        train_dataset=dataset['train'],
        eval_dataset=dataset['valid'],
        processing_class=tokenizer,     # LLM tokenizer / renamed : tokenizer -> processing_class from trl 0.12.0
        args=training_args,
        data_collator=collator
    )

    start_at = time.time()
    trainer.train()
    elapsed_time = time.time() - start_at

    used_memory = torch.cuda.memory_allocated() / (1024 * 1024)
    print(f'used_memory (LoRA rank = {lora_rank}) : {used_memory:.2f} MB')

    fine_tuning_test_result_dict['lora_rank'].append(lora_rank)
    fine_tuning_test_result_dict['elapsed_time'].append(elapsed_time)

    trainer.save_model('model_output')

trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


Converting train dataset to ChatML:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,3.593300
2,4.135700
3,3.936900
4,2.500700
5,2.597100
6,1.931100
7,1.791900
8,1.688200
9,1.140400
10,1.762600


used_memory (LoRA rank = 64) : 5968.73 MB


In [26]:
# save Fine-Tuning test result

fine_tuning_test_result_df = pd.DataFrame(fine_tuning_test_result_dict)

llm_name = llm_path.split('/')[-1]
fine_tuning_test_result_df.to_csv(f'fine_tuning_test_result_{llm_name}.csv')

In [27]:
fine_tuning_test_result_df

,lora_rank,elapsed_time
0,16,58.710687
1,32,62.962578
2,64,67.024919
